In [29]:
import numpy as np
from keras import *
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, Activation
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers as optim
from keras.models import load_model

In [30]:
(xtr,ytr),(xtest,ytest)=cifar100.load_data(label_mode="coarse")

169017344/169001437 [==============================] - 155s 1us/step


In [31]:
#Data Preprocessing (Zero Centering and Normalization)
ytr=to_categorical(ytr,20)
ytest=to_categorical(ytest,20)
xval=xtr[40000:]
yval=ytr[40000:]
xtr=xtr[0:40000]
ytr=ytr[0:40000]
xtr = np.array(xtr,dtype=np.float64)
xtr -= np.mean(xtr)
xtr /= np.std(xtr)
xval = np.array(xval,dtype=np.float64)
xval -= np.mean(xval)
xval /= np.std(xval)
xtest = np.array(xtest,dtype=np.float64)
xtest -= np.mean(xtest)
xtest /= np.std(xtest)
np.random.seed(5)

In [45]:
datagenerator = ImageDataGenerator(
                featurewise_center = False,
                samplewise_center = False,
                featurewise_std_normalization = False,
                samplewise_std_normalization = False,
                zca_whitening = False,
                rotation_range = 10,
                width_shift_range= 0.1,
                height_shift_range = 0.1,
                horizontal_flip = True,
                vertical_flip = False)

model = Sequential()
model.add(Flatten(input_shape=xtr.shape[1:]))

model.add(Dense(1500))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(750))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(300))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(20,activation='softmax',kernel_regularizer=regularizers.l2(0.01)))


op = optim.Adam(lr=1e-3,epsilon=1e-8)

model.compile(loss='categorical_crossentropy', optimizer=op,metrics=['accuracy'])

model.summary()

model.fit_generator(datagenerator.flow(xtr, ytr, batch_size=1500), steps_per_epoch=xtr.shape[0]//128,
                    epochs = 50,
                    validation_data=(xval, yval),
                    workers=4
                   )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_107 (Dense)            (None, 1500)              4609500   
_________________________________________________________________
activation_80 (Activation)   (None, 1500)              0         
_________________________________________________________________
batch_normalization_80 (Batc (None, 1500)              6000      
_________________________________________________________________
dropout_80 (Dropout)         (None, 1500)              0         
_________________________________________________________________
dense_108 (Dense)            (None, 750)               1125750   
_________________________________________________________________
activation_81 (Activation)   (None, 750)               0         
__________

In [46]:

scores = model.evaluate(xtest,ytest,batch_size=1500)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

10000/10000 [==============================] - 0s 21us/step
acc: 52.19%


In [47]:
model.save("./kerasimp.hdf5")